# Generating 90s Pop Lyrics at the Character level

## Goal
Generate 1 line of lyrics in the style of 90s Pop.

## Problem Formulation
X: Examples of a line of lyrics for the model to use (n_examples, max_length, n_characters)
Y: A generated sequence of characters that ends with <EOS> (n_examples, n_characters)
    
<EOS> will be a special character in the vocabulary which the model will use to know that it can stop predicting.

## Methodology
To accomplish this, we need:
1. Dataset: A corpus of 90s Pop lyrics
2. Vocabulary: A set of characters which will be used for generating lyrics
3. Model: A model which can encode the probability of the next character given a sequence of characters
4. Generate Lyrics: Use the model and an input to generate new lyrics

In [1]:
from datetime import datetime
from IPython.display import SVG
from keras import backend as K
from keras.callbacks import EarlyStopping, TensorBoard
from keras.layers import Dense, Input, LSTM
from keras.models import Model
from keras.optimizers import RMSprop
from keras.preprocessing.text import text_to_word_sequence
from keras.utils import plot_model
from keras.utils.vis_utils import model_to_dot
import numpy as np
import os
import pandas as pd
from random import sample
import sys

/home/peter/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# hyper parameters
activations = 128
batch_size = 50
epochs = 10
learning_rate = 0.01
max_char_n = 40
n_training = 400000
training_ratio = 0.3

In [3]:
# generate logging variables
variant = '{}ex-{}a-{}b-{}c'.format(n_training, activations, batch_size, max_char_n)
timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
log_dir = 'logs/{}-{}'.format(variant, timestamp)
diagram_dir = 'diagram/{}-{}'.format(variant, timestamp)

In [4]:
# to use GPU
os.environ["CUDA_VISIBLE_DEVICES"]="0"

# verify that a gpu is listed
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

## Functions

In [5]:
# Transforms a csv into an array of song lyrics (None, 1)
def csvToSongLyricsArray(csv):
    # filter for only lyrics from the 1990s, of the pop genre, and not instrumentals
    mask = (csv['year'] > 1989) & (csv['year'] < 2000) & (csv['genre'] == 'Pop') & (csv['lyrics'] != '[Instrumental]')
    filtered = csv[mask]
    
    # remove null values
    nonNull = filtered.dropna()
    
    # trim all the extra data. We only want the lyrics
    lyrics = nonNull['lyrics']
    
    # reindex the lyrics to make it easier to work with
    reindexed = lyrics.reset_index(drop=True)
    
    # lowercase the lyrics
    lowercased = reindexed[:].str.lower()
    
    # get the number of song lyrics
    n_songs = lowercased.shape[0]
    
    return lowercased, n_songs

In [6]:
def deprocessPrediction(ix_to_char, prediction):
    index = np.argmax(prediction)
    char = ix_to_char[index]
    
    return char

In [7]:
# filter out any song where lyrics contain a character outside the chars set
def filterLyrics(chars, lyrics):
    filtered_lyrics = []
    
    # for each song
    for lyric in lyrics:
        check = 0
        
        # split the lyric into an array of characters
        lyric_chars = list(lyric)
        
        # for each character, check if it's not in the chars set
        for char in lyric_chars:
            if char not in chars:
                check = 1

        # if all character are in the chars set
        # add it to our filter lyrics list
        if check == 0:
            filtered_lyrics.append(lyric_chars)
            
    # get the number of lyrics
    n_filtered_lyrics = len(filtered_lyrics)    
    
    return filtered_lyrics, n_filtered_lyrics

In [8]:
# flatten the previous into a list of song lyrics lines
def flatten_lyrics(lyrics):
    flattened_lyrics = [line for song in lyrics for line in song]
    n_chars = len(flattened_lyrics)
    
    return flattened_lyrics, n_chars

In [9]:
def generateCharacterConverters(chars):
    char_to_ix = { ch:i for i,ch in enumerate(sorted(chars)) }
    ix_to_char = { i:ch for i,ch in enumerate(sorted(chars)) }
    
    return char_to_ix, ix_to_char

In [10]:
def generateDataset(lyrics, n_chars, n_examples, max_char_n):
    dataset = []
    max_index = n_chars - max_char_n
    start_indices = np.random.randint(0, max_index, size=n_examples)

    for start_index in start_indices:
        end_index = start_index + max_char_n
        example = lyrics[start_index:end_index]
        start_index = end_index
        dataset.append(example)
        
    return dataset

In [11]:
def generateXYDatasets(char_to_ix, dataset, n_examples, n_chars_set, max_char_n):
    # create training input
    X_training = np.zeros((n_examples, max_char_n-1, n_chars_set), dtype='float32')
    X_training.shape
    
    # create training input
    Y_training = np.zeros((n_examples, n_chars_set), dtype='float32')
    Y_training.shape
    
    # fill input training set with word sequences, where words are one-hot encoded
    for li, line in enumerate(dataset):
        for ci, char in enumerate(line[:-1]):
            index = char_to_ix[char]
            X_training[li][ci][index] = 1
            
    # create training output
    for li, line in enumerate(dataset):
        char = line[-1]
        index = char_to_ix[char]
        Y_training[li][index] = 1
        
    return X_training, Y_training

In [12]:
def preprocessExample(char_to_ix, example, n_chars_set):
    chars = list(example)
    n_sample_chars = len(chars)

    preprocessed_example = np.zeros((1, n_sample_chars, n_chars_set), dtype='float32')

    for ci, char in enumerate(chars):
        index = char_to_ix[char]
        preprocessed_example[0][ci][index] = 1

    return preprocessed_example

In [13]:
def sample_predictions(preds, temperature=0.5):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return probas

## Extract and Transform Raw Dataset

In [14]:
# load raw data file as a dataframe
raw_data = pd.read_csv('data/raw.csv')

In [15]:
# get formatted_lyrics and number of songs
lyrics, n_lyrics = csvToSongLyricsArray(raw_data)

In [16]:
lyrics.head(10)

0    come they told me, pa rum pum pum pum\na new b...
1    over the ground lies a mantle, white\na heaven...
2    i just came back from a lovely trip along the ...
3    i'm dreaming of a white christmas\njust like t...
4    just hear those sleigh bells jingle-ing, ring-...
5    little rump shaker she can really shake and ba...
6    girl you want to sex me\ngirl, why don't you l...
7    oooh, tonight i want to turn the lights down l...
8    so you say he let you on, you'll never give yo...
9    something about you baby\nthat makes me wanna ...
Name: lyrics, dtype: object

In [17]:
# examine the number of song lyrics we have
print("Number of lyrics: {}".format(n_lyrics))
print("Lyric Example: {}".format(lyrics[0]))

Number of lyrics: 964
Lyric Example: come they told me, pa rum pum pum pum
a new born king to see, pa rum pum pum pum
our finest gifts we bring, pa rum pum pum pum
to lay before the king, pa rum pum pum pum,
rum pum pum pum, rum pum pum pum,
so to honor him, pa rum pum pum pum,
when we come.
little baby, pa rum pum pum pum
i am a poor boy too, pa rum pum pum pum
i have no gift to bring, pa rum pum pum pum
that's fit to give the king, pa rum pum pum pum,
rum pum pum pum, rum pum pum pum,
shall i play for you, pa rum pum pum pum,
on my drum?
mary nodded, pa rum pum pum pum
the ox and lamb kept time, pa rum pum pum pum
i played my drum for him, pa rum pum pum pum,
rum pum pum pum, rum pum pum pum,
then he smiled at me, pa rum pum pum pum
me and my drum
me and my drum
me and my drum
me and my drum
come they told me, pa rum pum pum pum
me and my drum


## Filter out non-english lyrics

In [18]:
chars = [' ', "'", 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'x', 'z', '\n', '!', '"', '$', '%', '&', '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~']

In [19]:
# determine number of charecters in our set
n_chars_set = len(chars)
print("Number of characters in chars: {}".format(n_chars_set))

Number of characters in chars: 60


In [20]:
# filter out any song where lyrics contain a character outside the english set
filtered_lyrics, n_filtered_lyrics = filterLyrics(chars, lyrics)

In [21]:
# examine the resulting number of song lyrics lines we have
print("Number of english songs: {}".format(n_filtered_lyrics))
print("A english song lyric: {}".format(filtered_lyrics[0]))

Number of english songs: 782
A english song lyric: ['c', 'o', 'm', 'e', ' ', 't', 'h', 'e', 'y', ' ', 't', 'o', 'l', 'd', ' ', 'm', 'e', ',', ' ', 'p', 'a', ' ', 'r', 'u', 'm', ' ', 'p', 'u', 'm', ' ', 'p', 'u', 'm', ' ', 'p', 'u', 'm', '\n', 'a', ' ', 'n', 'e', 'w', ' ', 'b', 'o', 'r', 'n', ' ', 'k', 'i', 'n', 'g', ' ', 't', 'o', ' ', 's', 'e', 'e', ',', ' ', 'p', 'a', ' ', 'r', 'u', 'm', ' ', 'p', 'u', 'm', ' ', 'p', 'u', 'm', ' ', 'p', 'u', 'm', '\n', 'o', 'u', 'r', ' ', 'f', 'i', 'n', 'e', 's', 't', ' ', 'g', 'i', 'f', 't', 's', ' ', 'w', 'e', ' ', 'b', 'r', 'i', 'n', 'g', ',', ' ', 'p', 'a', ' ', 'r', 'u', 'm', ' ', 'p', 'u', 'm', ' ', 'p', 'u', 'm', ' ', 'p', 'u', 'm', '\n', 't', 'o', ' ', 'l', 'a', 'y', ' ', 'b', 'e', 'f', 'o', 'r', 'e', ' ', 't', 'h', 'e', ' ', 'k', 'i', 'n', 'g', ',', ' ', 'p', 'a', ' ', 'r', 'u', 'm', ' ', 'p', 'u', 'm', ' ', 'p', 'u', 'm', ' ', 'p', 'u', 'm', ',', '\n', 'r', 'u', 'm', ' ', 'p', 'u', 'm', ' ', 'p', 'u', 'm', ' ', 'p', 'u', 'm', ',', ' ', 'r',

In [22]:
# flatten english song lyrics
flattened_lyrics, n_chars = flatten_lyrics(filtered_lyrics)

In [23]:
print("Number of song lyrics characters: {}".format(n_chars))
print("Section of song lyrics: {}".format(flattened_lyrics[0:100]))

Number of song lyrics characters: 846434
Section of song lyrics: ['c', 'o', 'm', 'e', ' ', 't', 'h', 'e', 'y', ' ', 't', 'o', 'l', 'd', ' ', 'm', 'e', ',', ' ', 'p', 'a', ' ', 'r', 'u', 'm', ' ', 'p', 'u', 'm', ' ', 'p', 'u', 'm', ' ', 'p', 'u', 'm', '\n', 'a', ' ', 'n', 'e', 'w', ' ', 'b', 'o', 'r', 'n', ' ', 'k', 'i', 'n', 'g', ' ', 't', 'o', ' ', 's', 'e', 'e', ',', ' ', 'p', 'a', ' ', 'r', 'u', 'm', ' ', 'p', 'u', 'm', ' ', 'p', 'u', 'm', ' ', 'p', 'u', 'm', '\n', 'o', 'u', 'r', ' ', 'f', 'i', 'n', 'e', 's', 't', ' ', 'g', 'i', 'f', 't', 's', ' ', 'w', 'e']


## Extract the subset we are interested in

In [24]:
# generate n_training example of max_char_n length
dataset = generateDataset(flattened_lyrics, n_chars, n_training, max_char_n)
print("Number of examples in dataset: {}".format(len(dataset)))
print("Example: {}".format(dataset[0]))

Number of examples in dataset: 400000
Example: ['d', 'o', 'r', ' ', 'a', 'l', 'l', ' ', 'e', 'n', ' ', 'l', 'a', ' ', 'm', 'i', 'n', 'a', '\n', 'y', ' ', 'a', 'r', 'r', 'a', 'n', 'c', 'a', 'n', 'd', 'o', ' ', 'n', 'e', 'g', 'r', 'o', ' ', 'c', 'a']


In [25]:
# create dictionarys
char_to_ix, ix_to_char = generateCharacterConverters(chars)
print("ix_to_char: {}".format(ix_to_char))
print("char_to_ix: {}".format(char_to_ix))

ix_to_char: {0: '\n', 1: ' ', 2: '!', 3: '"', 4: '$', 5: '%', 6: '&', 7: "'", 8: '(', 9: ')', 10: '*', 11: '+', 12: ',', 13: '-', 14: '.', 15: '/', 16: ':', 17: ';', 18: '<', 19: '=', 20: '>', 21: '?', 22: '@', 23: '[', 24: '\\', 25: ']', 26: '^', 27: '_', 28: '`', 29: 'a', 30: 'b', 31: 'c', 32: 'd', 33: 'e', 34: 'f', 35: 'g', 36: 'h', 37: 'i', 38: 'j', 39: 'k', 40: 'l', 41: 'm', 42: 'n', 43: 'o', 44: 'p', 45: 'q', 46: 'r', 47: 's', 48: 't', 49: 'u', 50: 'v', 51: 'w', 52: 'x', 53: 'x', 54: 'y', 55: 'z', 56: '{', 57: '|', 58: '}', 59: '~'}
char_to_ix: {'\n': 0, ' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, '&': 6, "'": 7, '(': 8, ')': 9, '*': 10, '+': 11, ',': 12, '-': 13, '.': 14, '/': 15, ':': 16, ';': 17, '<': 18, '=': 19, '>': 20, '?': 21, '@': 22, '[': 23, '\\': 24, ']': 25, '^': 26, '_': 27, '`': 28, 'a': 29, 'b': 30, 'c': 31, 'd': 32, 'e': 33, 'f': 34, 'g': 35, 'h': 36, 'i': 37, 'j': 38, 'k': 39, 'l': 40, 'm': 41, 'n': 42, 'o': 43, 'p': 44, 'q': 45, 'r': 46, 's': 47, 't': 48, 'u': 49, 

## X and Y Datasets

In [26]:
# create training input and output
X_training, Y_training = generateXYDatasets(char_to_ix, dataset, n_training, n_chars_set, max_char_n)

## Validate Dataset

In [27]:
x_example = X_training[2] 

x_training_string = []
for woh in x_example:
    char = deprocessPrediction(ix_to_char, woh)
    x_training_string.append(char)
x_training_string_formatted = "".join(x_training_string)
print("X_training shape: {}".format(x_example.shape))
print("X_training example one-hot: {}".format(x_example))
print("X_training example: {}".format(x_training_string_formatted))

X_training shape: (39, 60)
X_training example one-hot: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
X_training example: od to you girl
i'll be good to you babe


In [28]:
y_example = Y_training[2]

char = deprocessPrediction(ix_to_char, y_example)
print("Y_training shape: {}".format(y_example.shape))
print("Y_training example one-hot: {}".format(y_example))
print("Y_training example: {}".format(char))

Y_training shape: (60,)
Y_training example one-hot: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Y_training example: 



## Model

In [29]:
model_input = Input(shape=(None, n_chars_set))
x = LSTM(activations)(model_input)
x = Dense(n_chars_set, activation='softmax')(x)

In [30]:
model = Model(inputs=model_input, outputs=x)

optimizer = RMSprop(lr=learning_rate)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [31]:
# set up callbacks
early = EarlyStopping(monitor='val_acc',
                      min_delta=0,
                      patience=10,
                      verbose=1,
                      mode='auto')

In [ ]:
model.fit(X_training, 
          Y_training, 
          batch_size=batch_size, 
          epochs=epochs, 
          shuffle=True,
          validation_split=training_ratio,
          callbacks=[early, TensorBoard(log_dir=log_dir)])

Train on 280000 samples, validate on 120000 samples
Epoch 1/10
280000/280000 [==============================] - 402s 1ms/step - loss: 1.7871 - acc: 0.4684 - val_loss: 1.6223 - val_acc: 0.5134
Epoch 2/10
280000/280000 [==============================] - 447s 2ms/step - loss: 1.5746 - acc: 0.5265 - val_loss: 1.5616 - val_acc: 0.5319
Epoch 3/10
279950/280000 [============================>.] - ETA: 0s - loss: 1.5234 - acc: 0.5431

## Make a prediction

In [ ]:
example = 'sweet dreams are made of these'

In [ ]:
# convert example to a sequence of one-hot encoded chars
preprocessed_example = preprocessExample(char_to_ix, example, n_chars_set)

In [ ]:
prediction = model.predict(preprocessed_example)

In [ ]:
char = deprocessPrediction(ix_to_char, prediction[0])
print("Prediction: {}".format(char))

## Generate a sequence from a sequence

In [ ]:
sys.stdout.write(example)

for i in range(500):
    prediction = model.predict(preprocessed_example, verbose=0)[0]
    sampled_prediction = sample_predictions(prediction)
    next_char = deprocessPrediction(ix_to_char, sampled_prediction[0])
    preprocessed_example[0][:-1] = preprocessed_example[0][1:]
    preprocessed_example[0][-1] = sampled_prediction
    sys.stdout.write(next_char)
    sys.stdout.flush()

## Plot model

In [ ]:
# draw a model diagram and save it to disk
plot_model(model, to_file=diagram_dir)
SVG(model_to_dot(model).create(prog='dot', format='svg'))